In [1]:
import numpy as np
import pandas as pd

### Q1. Embedding the query

In [2]:
from fastembed import TextEmbedding

In [3]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [4]:
model = TextEmbedding(model_name=model_handle)
query = 'I just discovered the course. Can I join now?'
embedding_query = list(model.embed(query))

In [5]:
min_val = np.array(embedding_query[0]).min()

print(f"Minimal value in the embedding array: {min_val}")


Minimal value in the embedding array: -0.11726373885183883


### Q2. Cosine similarity with another vector

In [6]:
doc = 'Can I still join the course after the start date?'

In [7]:
embedding_doc = list(model.embed(doc))

In [8]:
embedding_query = np.array(list(model.embed(query))[0])
embedding_doc = np.array(list(model.embed(doc))[0])

In [9]:
# Nomalizing the vectors 
embedding_query /= np.linalg.norm(embedding_query)
embedding_doc /= np.linalg.norm(embedding_doc)

In [10]:
cosine_similarity = embedding_query.dot(embedding_doc)

print("The Cosine Similarity between the embeddings is:", cosine_similarity)

The Cosine Similarity between the embeddings is: 0.9008528895674548


### Q3. Ranking by cosine

In [11]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [12]:
V = np.array([list(model.embed(doc["text"]))[0] for doc in documents])
V = V / np.linalg.norm(V, axis=1, keepdims=True)  # L2-normalize each row

In [13]:
cos_similarities = V.dot(embedding_query)

In [14]:
print("Most similar document index:", int(np.argmax(cos_similarities)))

Most similar document index: 1


### Q4. Ranking by cosine, version two

In [16]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
V_full = np.array([list(model.embed(text))[0] for text in full_texts])
V_full = V_full / np.linalg.norm(V_full, axis=1, keepdims=True)

In [17]:
cos_similarities = V.dot(embedding_query)

In [18]:
print("Most similar document index:", int(np.argmax(cos_similarities)))

Most similar document index: 1


No , in other cases it could be different because:

    Including the question adds more semantic cues and contextual alignment.

    If a text is vague but the question clarifies its intent, the model may now recognize a better match.

    Conversely, a noisy or unrelated question might lower the similarity.

### Q5. Selecting the embedding model

In [19]:
models = TextEmbedding.list_supported_models()

In [20]:
min_dim = min(model["dim"] for model in models)
print("Smallest embedding dimension available:", min_dim)

Smallest embedding dimension available: 384


### Q6. Indexing with qdrant

In [21]:
model = TextEmbedding(model_name="BAAI/bge-small-en")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

c:\Users\Siddharth\Downloads\LLM Zoomcamp - 2025\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Siddharth\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [22]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

```bash docker run -p 6333:6333 -p 6334:6334 -v "${PWD}\qdrant_storage:/qdrant/storage" qdrant/qdrant```

In [23]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [ ]:
# Define the collection name
collection_name = "zoomcamp-homework2"
# Delete the collection if it already exsists
client.delete_collection(collection_name=collection_name)
# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [26]:
from uuid import uuid4

# Prepare data for indexing
payloads = []
vectors = []
ids = []

for doc in documents:
    # Combine question + text
    full_text = doc["question"] + " " + doc["text"]

    # Embed
    embedding = list(model.embed(full_text))[0]

    # Save data
    vectors.append(embedding)
    payloads.append({
        "course": doc["course"],
        "question": doc["question"],
        "text": doc["text"]
    })
    ids.append(str(uuid4()))  # Unique ID for each document

In [27]:
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=ids[i],
            vector=vectors[i],
            payload=payloads[i]
        )
        for i in range(len(vectors))
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [32]:
query = "I just discovered the course. Can I join now?"

In [43]:
def search(query, limit=1):
    # Embed the query
    query_vector = list(model.embed(query))[0]

    results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=limit,
        with_payload=True
    )

    return results


In [44]:
results = search(query)
top_score = results[0].score
print("Highest score from search result:", top_score)

Highest score from search result: 0.8703172


C:\Users\Siddharth\AppData\Local\Temp\ipykernel_17472\3366497221.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
